# Probe Data - Map Matching
## Nick Paras | Kapil Garg

### Assignment 2

Input: Probe data and map [probe_data_map_matching.rar](https://canvas.northwestern.edu/courses/51440/files/3334329/download?wrap=1)

-The raw probe points in Germany collected in 9 months

-The link data for the links that probe points can be map-matched to.

Tasks:
-- map match probe points to road links

-- derive road slope for each road link

-- evaluate the derived road slope with the surveyed road slope in the link data file

**Please submit your code and slides presentation of your approach and results including evaluation comparing with the slopes in the link data file**

### Setup

We use **Python 3.6** and rely on the dependencies:
* numpy
* scikit-learn
* matplotlib
* pandas

We also use Jupyter Notebooks for our code and reports. For quick setup, please create a conda environment with the following:

    $ conda create --name probe-data pandas matplotlib numpy scikit-learn

and then activate the conda environment with

    $ source activate probe-data


In [7]:
# Imports
import os
import math
import csv
import operator
import multiprocessing as mp
import itertools

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from datetime import datetime
from haversine import haversine
from functools import reduce

import link_classes as lc
import graph_classes as graph

%matplotlib inline

# Constants
DATA_DIR = '../data'

In [8]:
# Utility functions
def bearing(start, end):
    """
    Computes the bearing in degrees between two geopoints
    
    Inputs:
        start (tuple of lat, long): starting geolocation
        end (tuple of lat, long): ending geolocation
    
    Outputs:
        (float): bearing in degrees between start and end
    """
    phi_1 = math.radians(start[0])
    phi_2 = math.radians(end[0])
    lambda_1 = math.radians(start[1])
    lambda_2 = math.radians(end[1])
    
    x = math.cos(phi_2) * math.sin(lambda_2 - lambda_1)
    y = math.cos(phi_1) * math.sin(phi_2) - (math.sin(phi_1) * math.cos(phi_2) * math.cos(lambda_2 - lambda_1))

    return (math.degrees(math.atan2(x, y)) + 360) % 360

## Loading Probe Data for Map Matching

Here we'll load our data from the two csv's into Pandas DataFrames.

In [9]:
probe_headers = ['sampleID', 
                 'dateTime', 
                 'sourceCode', 
                 'latitude', 
                 'longitude', 
                 'altitude', 
                 'speed', 
                 'heading']

probe_data = pd.read_csv(os.path.join(DATA_DIR, 'Partition6467ProbePoints.csv'), header=None, names=probe_headers)
probe_data.head()

,sampleID,dateTime,sourceCode,latitude,longitude,altitude,speed,heading
0,3496,6/12/2009 6:12:49 AM,13,51.496868,9.386022,200,23,339
1,3496,6/12/2009 6:12:54 AM,13,51.496682,9.386157,200,10,129
2,3496,6/12/2009 6:12:59 AM,13,51.496705,9.386422,201,21,60
3,3496,6/12/2009 6:13:04 AM,13,51.496749,9.386840,201,0,360
4,3496,6/12/2009 6:13:09 AM,13,51.496864,9.387294,199,0,360


In [10]:
link_headers = ['linkPVID', 
                'refNodeID', 
                'nrefNodeID', 
                'length', 
                'functionalClass', 
                'directionOfTravel', 
                'speedCategory', 
                'fromRefSpeedLimit', 
                'toRefSpeedLimit', 
                'fromRefNumLanes', 
                'toRefNumLanes', 
                'multiDigitized', 
                'urban', 
                'timeZone', 
                'shapeInfo', 
                'curvatureInfo', 
                'slopeInfo']

# load raw link data
link_data = pd.read_csv(os.path.join(DATA_DIR, 'Partition6467LinkData.csv'), header=None, names=link_headers)
link_data.head()

,linkPVID,refNodeID,nrefNodeID,length,functionalClass,directionOfTravel,speedCategory,fromRefSpeedLimit,toRefSpeedLimit,fromRefNumLanes,toRefNumLanes,multiDigitized,urban,timeZone,shapeInfo,curvatureInfo,slopeInfo
0,62007637,162844982,162809070,335.04,5,B,7,30,30,0,0,F,T,0.0,51.4965800/9.3862299/|51.4994700/9.3848799/,NaN,NaN
1,567329767,162844982,162981512,134.56,5,B,7,0,0,0,0,F,T,0.0,51.4965800/9.3862299/|51.4966899/9.3867100/|51...,NaN,NaN
2,62007648,162877732,162844982,97.01,5,B,7,30,30,0,0,F,T,0.0,51.4962899/9.3849100/|51.4965800/9.3862299/,NaN,NaN
3,78670326,162877732,163152693,314.84,5,B,7,30,30,0,0,F,T,0.0,51.4962899/9.3849100/|51.4990000/9.3836099/,NaN,NaN
4,51881672,174713859,174587951,110.17,3,B,6,50,50,2,2,F,T,0.0,53.0643099/8.7903400/45.79|53.0650299/8.791470...,NaN,0.00/-0.090|110.17/0.062


In [11]:
# create link data lookup dictionary
links = []
link_db = lc.LinkDatabase()
with open(os.path.join(DATA_DIR, 'Partition6467LinkData.csv'), 'r') as csvfile:
    rdr = csv.DictReader(csvfile, delimiter=',', fieldnames=link_headers)
    for r in rdr:
        rl = lc.RoadLink(r)
        links.append(rl)
        link_db.insert_link(rl)

In [14]:
# create road link graph
node_id_map = {}
with open(os.path.join(DATA_DIR, 'Partition6467LinkData.csv'), 'r') as csvfile:
    rdr = csv.DictReader(csvfile, delimiter=',', fieldnames=link_headers)
    for r in rdr:
        try:
            node_id_map[r['refNodeID']] += r['linkPVID']
        except KeyError:
            node_id_map[r['refNodeID']] = [r['linkPVID']]

road_graph = graph.RoadNetwork(node_id_map)
with open(os.path.join(DATA_DIR, 'Partition6467LinkData.csv'), 'r') as csvfile:
    rdr = csv.DictReader(csvfile, delimiter=',', fieldnames=link_headers)
    for r in rdr:
        road_graph.insert(graph.RoadNetworkNode(node_id_map, \
                                                [[float(j) for j in i.split('/')[:2]] for i in r['shapeInfo'].split('|')], \
                                                r['linkPVID'], \
                                                r['nrefNodeID']))

## ST-Matching Algorithm
Below is an implementation of the ST-Matching algorithm from the [Map-Matching for Low_sampling-Rate GPS Trajectories](https://www.microsoft.com/en-us/research/publication/map-matching-for-low-sampling-rate-gps-trajectories/?from=http%3A%2F%2Fresearch.microsoft.com%2Fapps%2Fpubs%2Fdefault.aspx%3Fid%3D105051) paper. 

### Defining useful utility functions

In [15]:
def nearest_n_segments(lat, long, n):
    """
    Uses link_db to find nearest n road segments
    
    Inputs:
        lat (float): latitude of probe point
        lon (float): longitude of probe point
        n (int): number of roads road segments to return
        
    Output: 
        (list of tuples): (linkPVID, distance) of n-nearest road segments 
        
    """
    # find nearest n links
    output = []
    try:
        link_search = [(x, haversine(x.refLatLon, (lat, long))) for x in link_db.get_links(lat, long)]
        link_search.sort(key=operator.itemgetter(1))
        link_search = link_search[0:n]

        # extract only link PVIDs from search
        output = [(int(x[0].linkPVID), x[1]) for x in link_search]
    except KeyError:
        pass
    
    return output

### Defining scoring functions

### Main Algorithm

### Putting it all together

In [40]:
# time code
import time
t0 = time.time()

# sample only first sample_size to make computation faster
sample_size = 1000
# sample_size = len(probe_data) # for all data

# add road link
probe_data['linkPVID'] = 0

# parallelizable function
def link_road_parallel(indicies):
    """
    Links road to probe for set of indicies
    
    Input:
        indicies (list of floats): indicies to find nearest link for
    """
    output = [(0, 0) for x in range(indicies[1] - indicies[0])]
    n = 3
    counter = 0
    for row in probe_data[indicies[0]:indicies[1]].itertuples():
        output[counter] = (row.Index, closest_by_heading(nearest_n_segments(row.latitude, row.longitude, n),
                                                         row.heading))
        counter += 1
    
    return output

# run in parallel
N_CORES = mp.cpu_count()
C_SIZE = math.ceil(sample_size / N_CORES)

pool = mp.Pool(N_CORES)
r = pool.map(link_road_parallel, [[(C_SIZE * i), ((i + 1) * C_SIZE)] for i in range(N_CORES)])
linkings = list(itertools.chain.from_iterable(r))

# assign values to probe_data
stacked_values = np.dstack(linkings)[0]
probe_data.loc[stacked_values[0], 'linkPVID'] = stacked_values[1]
        
# finish timing
t1 = time.time()
print(str((t1 - t0) / 60) + ' minutes for ' + str(sample_size) + ' data points using ' + str(N_CORES) + ' CPU threads.')

0.2601946830749512 minutes for 1000 data points using 8 CPU threads.


In [14]:
probe_data[0:sample_size].to_csv('./sample_linked_data.csv', index=False)

### Create output file
The output file has the following columns (columns in **bold** are pulled from the LinkData csv):
- sampleID: a unique identifier for the set of probe points that were collected from a particular phone.
- dateTime: date and time that the probe point was collected.
- sourceCode: a unique identifier for the data supplier (13 = Nokia).
- latitude: latitude in decimal degrees.
- longitude: longitude in decimal degrees.
- altitude: altitude in meters.
- speed: speed in KPH.
- heading: heading in degrees.
- linkPVID: published versioned identifier for the link.
- **direction**: direction the vehicle was travelling on thelink (F = from ref node, T = towards ref node).
- **distFromRef**: distance from the reference node to the map-matched probe point location on the link in decimal meters.
- **distFromLink**: perpendicular distance from the map-matched probe point location on the link to the probe point in decimal meters.